Piotr Ozimek's Retina implementation taken from https://github.com/Pozimek/RetinaVision.

In [ ]:
"""
This file wraps the shared libraries compiled from the GPU code into
Python objects, serving as proxies to the main classes.
"""
import sys
import numpy as np
from os import chdir, getcwd, path, listdir
np.seterr(divide='ignore', invalid='ignore')

import ctypes

lib_path = '/content/drive/My Drive/Colab Notebooks/RetinaSmartCamera/bin/linux/libRetinaCUDA.so'

lib = ctypes.cdll.LoadLibrary(lib_path)


def convert_from_gpu(rgb_image_vector):
    '''
    Reshape flat RGB image vector from GPU to fit original implementation.\n
    Parameters
    ----------
    rgb_image_vector : np.ndarray
        Must be flat, with length of retina_size * 3\n
    Returns
    -------
    rgb_image_vector : np.ndarray
        image vector shaped [retina_size, 3]
    '''
    retina_size = int(len(rgb_image_vector) / 3)
    return np.hstack(\
        (np.resize(rgb_image_vector[0:retina_size], (retina_size,1)),\
        np.resize(rgb_image_vector[retina_size:2*retina_size], (retina_size,1)), \
        np.resize(rgb_image_vector[2*retina_size:3*retina_size], (retina_size,1))))

def convert_to_gpu(rgb_image_vector):
    '''
    Flattens RGB image vector to become compatible with GPU computation.\n
    Parameters
    ----------
    rgb_image_vector : np.ndarray
        must have shape of [retina_size, 3]\n
    Returns
    -------
    rgb_image_vector : np.ndarray
        flattened image vector
    '''
    retina_size = rgb_image_vector.shape[0]
    return np.append(\
        np.resize(rgb_image_vector[:,0], (1, retina_size))[0],\
        [np.resize(rgb_image_vector[:,1], (1, retina_size))[0],\
        np.resize(rgb_image_vector[:,2], (1, retina_size))[0]])


class CudaRetina(object):
    def resolveError(self, err):
        if err == -1:
            raise Exception("Invalid arguments")
        elif err == 1:
            raise Exception("Retina was not initialized properly")
        elif err == 2:
            raise Exception("Retina size did not match the parameter")
        elif err == 3:
            raise Exception("Image parameteres did not match")

    def __init__(self):
        lib.Retina_new.argtypes = []
        lib.Retina_new.restype = ctypes.c_void_p
        lib.Retina_delete.argtypes = [ctypes.c_void_p]
        lib.Retina_delete.restype = ctypes.c_void_p

        lib.Retina_setSamplingFields.argtypes = [ctypes.c_void_p, \
        ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_double), ctypes.c_size_t]
        lib.Retina_setSamplingFields.restype = ctypes.c_int
        
        lib.Retina_setGaussNormImage.argtypes = [ctypes.c_void_p, \
        ctypes.POINTER(ctypes.c_double), ctypes.c_size_t, ctypes.c_size_t, ctypes.c_size_t]
        lib.Retina_setGaussNormImage.restype = ctypes.c_int
        
        lib.Retina_getGaussNormImage.argtypes = [ctypes.c_void_p, \
        ctypes.POINTER(ctypes.c_double), ctypes.c_size_t, ctypes.c_size_t, ctypes.c_size_t]
        lib.Retina_getGaussNormImage.restype = ctypes.c_int
        
        lib.Retina_sample.argtypes = [ctypes.c_void_p, \
        ctypes.POINTER(ctypes.c_uint8), ctypes.c_size_t, ctypes.c_size_t, ctypes.c_size_t, \
        ctypes.POINTER(ctypes.c_double), ctypes.c_size_t, ctypes.c_bool]
        lib.Retina_sample.restype = ctypes.c_int

        lib.Retina_inverseAndNormalise.argtypes = [ctypes.c_void_p, \
        ctypes.POINTER(ctypes.c_double), ctypes.c_size_t, ctypes.POINTER(ctypes.c_uint8), \
        ctypes.c_size_t, ctypes.c_size_t, ctypes.c_size_t, ctypes.c_bool]
        lib.Retina_inverseAndNormalise.restype = ctypes.c_int

        lib.Retina_inverse.argtypes = [ctypes.c_void_p, \
        ctypes.POINTER(ctypes.c_double), ctypes.c_size_t, ctypes.POINTER(ctypes.c_double), \
        ctypes.c_size_t, ctypes.c_size_t, ctypes.c_size_t, ctypes.c_bool]
        lib.Retina_inverse.restype = ctypes.c_int
        
        lib.Retina_getRetinaSize.argtypes = [ctypes.c_void_p]
        lib.Retina_getRetinaSize.restype = ctypes.c_int

        lib.Retina_getImageHeight.argtypes = [ctypes.c_void_p]
        lib.Retina_getImageHeight.restype = ctypes.c_int
        lib.Retina_setImageHeight.argtypes = [ctypes.c_void_p, ctypes.c_int]
        lib.Retina_setImageHeight.restype = ctypes.c_void_p

        lib.Retina_getImageWidth.argtypes = [ctypes.c_void_p]
        lib.Retina_getImageWidth.restype = ctypes.c_int
        lib.Retina_setImageWidth.argtypes = [ctypes.c_void_p, ctypes.c_int]
        lib.Retina_setImageWidth.restype = ctypes.c_void_p

        lib.Retina_getRGB.argtypes = [ctypes.c_void_p]
        lib.Retina_getRGB.restype = ctypes.c_bool
        lib.Retina_setRGB.argtypes = [ctypes.c_void_p, ctypes.c_bool]
        lib.Retina_setRGB.restype = ctypes.c_void_p

        lib.Retina_getCenterX.argtypes = [ctypes.c_void_p]
        lib.Retina_getCenterX.restype = ctypes.c_int
        lib.Retina_setCenterX.argtypes = [ctypes.c_void_p, ctypes.c_int]
        lib.Retina_setCenterX.restype = ctypes.c_void_p
        
        lib.Retina_getCenterY.argtypes = [ctypes.c_void_p]
        lib.Retina_getCenterY.restype = ctypes.c_int
        lib.Retina_setCenterY.argtypes = [ctypes.c_void_p, ctypes.c_int]
        lib.Retina_setCenterY.restype = ctypes.c_void_p

        self.obj = lib.Retina_new()

    def __del__(self):
        '''Calls the C++ destructor on self'''
        lib.Retina_delete(self.obj)

    @property
    def retina_size(self):
        '''int, number of sampling fields in the retina'''
        return lib.Retina_getRetinaSize(self.obj)

    @property
    def image_height(self):
        '''int, height of the image the retina can process (input image)
        Setting the property will invalidate gauss norm image'''
        return lib.Retina_getImageHeight(self.obj)
    @image_height.setter
    def image_height(self, value):
        '''int, height of the image the retina can process (input image)
        Setting the property will invalidate gauss norm image'''
        return lib.Retina_setImageHeight(self.obj, value)

    @property
    def image_width(self):
        '''int, width of the image the retina can process (input image)
        Setting the property will invalidate gauss norm image'''
        return lib.Retina_getImageWidth(self.obj)
    @image_width.setter
    def image_width(self, value):
        '''int, width of the image the retina can process (input image)
        Setting the property will invalidate gauss norm image'''
        return lib.Retina_setImageWidth(self.obj, value)

    @property
    def rgb(self):
        '''bool, whether the retina can process rgb images (input image)
        Setting the property will invalidate gauss norm image'''
        return lib.Retina_getRGB(self.obj)
    @rgb.setter
    def rgb(self, value):
        '''bool, whether the retina can process rgb images (input image)
        Setting the property will invalidate gauss norm image'''
        return lib.Retina_setRGB(self.obj, value)

    @property
    def center_x(self):
        '''int, X coordinate of the retina center
        Note: in openCV this is [1]
        Setting the property will invalidate gauss norm image'''
        return lib.Retina_getCenterX(self.obj)
    @center_x.setter
    def center_x(self, value):
        '''int, X coordinate of the retina center
        Note: in openCV this is [1]
        Setting the property will invalidate gauss norm image'''
        return lib.Retina_setCenterX(self.obj, value)

    @property
    def center_y(self):
        '''int, Y coordinate of the retina center
        Note: in openCV this is [0]
        Setting the property will invalidate gauss norm image'''
        return lib.Retina_getCenterY(self.obj)
    @center_y.setter
    def center_y(self, value):
        '''int, Y coordinate of the retina center
        Note: in openCV this is [0]
        Setting the property will invalidate gauss norm image'''
        return lib.Retina_setCenterY(self.obj, value)
    
    def set_samplingfields(self, loc, coeff):
        '''
        Sets the sampling fields of the retina\n
        Parameters
        ----------
        loc : np.ndarray
            shape [retina_size, 7], 7 values each line, locations of the fields (from matlab)
        coeff : np.ndarray
            kernels of the sampling
        '''
        if loc.shape[0] != len(coeff.flatten()):
            print("Number of locs and coeffs must be the same")
            return
        loc1D = loc.flatten()
        coeff1D = []
        for i in coeff.flatten():
            coeff1D += i.flatten().tolist()

        #self.__retina_size = loc.shape[0]
        err = lib.Retina_setSamplingFields(self.obj, (ctypes.c_float * len(loc1D))(*loc1D),
                (ctypes.c_double * len(coeff1D))(*coeff1D), loc.shape[0])
        self.resolveError(err)

    def set_gauss_norm(self, gauss_norm=None, compute_on_gpu=False):
        '''
        Sets the gaussian matrix to normalise with on backprojection\n
        Parameters
        ----------
        guass_norm : np.ndarray, optional
            shape must be [image_height, image_width]
            if None, CUDA will generate the gauss norm
            if not None, height and width must match with retina's 
            (3rd dimension is handled by the function)
        compute_on_gpu : bool, optional
            indicates whether to compute gauss norm on the GPU,
            or copy the one passed as parameter 
        '''
        if compute_on_gpu:
            lib.Retina_setGaussNormImage(self.obj, None, 0, 0, 0)
        elif gauss_norm is not None:
            gauss_channels = 1
            gauss_norm_p = gauss_norm.flatten()
            if self.rgb:
                gauss_channels = 3#gauss_norm.shape[2]
                gauss_norm_p = np.vstack((gauss_norm[:,:].flatten(), gauss_norm[:,:].flatten(), gauss_norm[:,:].flatten()))

            err = lib.Retina_setGaussNormImage(self.obj, \
                    gauss_norm_p.ctypes.data_as(ctypes.POINTER(ctypes.c_double)), \
                    gauss_norm.shape[0], gauss_norm.shape[1], gauss_channels)
            self.resolveError(err)

    def sample(self, image):
        '''
        Sample image\n
        Parameters
        ----------
        image : np.ndarray
            height, width and rgb must match the retina parameters\n
        Returns
        -------
        image_vector : np.ndarray
            sampled flat image vector
            if rgb, must be reshaped to become compatible (convert_from_gpu)
        '''
        image_vector = np.empty(self.retina_size * (3 if self.rgb else 1), dtype=ctypes.c_double)

        image_channels = 1
        image_p = image.flatten()
        if self.rgb:
            image_channels = image.shape[2]
            image_p = np.vstack((image[:,:,0].flatten(), image[:,:,1].flatten(), image[:,:,2].flatten()))
        
        err = lib.Retina_sample(self.obj, image_p.ctypes.data_as(ctypes.POINTER(ctypes.c_uint8)), \
                image.shape[0], image.shape[1], image_channels, \
                image_vector.ctypes.data_as(ctypes.POINTER(ctypes.c_double)), \
                image_vector.shape[0], False)
        self.resolveError(err)

        return convert_from_gpu(image_vector) if self.rgb else image_vector

    def backproject(self, image_vector, normalise=True):
        '''
        Backprojects image from image vector\n
        Parameters
        ----------
        image_vector : np.ndarray
            length must match retina size\n
        normalise : bool
            Whether to normalise the image on the device.
            Note that this changes the return array type: true -> uint8, false -> double
            If true, gauss norm must be set on the device, otherwise the behaviour is undefined.
        Returns
        -------
        image : np.ndarray
            Backprojected image
        '''
        if len(image_vector.shape) > 1:
            image_vector = convert_to_gpu(image_vector)

        channels = (3 if self.rgb else 1)
        image = np.empty(self.image_height * self.image_width * channels, dtype=ctypes.c_uint8)

        err = 0
        if normalise:
            err = lib.Retina_inverseAndNormalise(self.obj, \
                image_vector.ctypes.data_as(ctypes.POINTER(ctypes.c_double)), \
                self.retina_size * channels, image.ctypes.data_as(ctypes.POINTER(ctypes.c_uint8)), \
                self.image_height, self.image_width, channels, False)
        else:
            image = np.empty(self.image_height * self.image_width * channels, dtype=ctypes.c_double)
            err = lib.Retina_inverse(self.obj, \
                image_vector.ctypes.data_as(ctypes.POINTER(ctypes.c_double)), \
                self.retina_size * channels, image.ctypes.data_as(ctypes.POINTER(ctypes.c_double)), \
                self.image_height, self.image_width, channels, False)
        self.resolveError(err)
        
        if self.rgb:
            flat_length = self.image_height * self.image_width
            out = np.dstack(\
            (np.resize(image[0:flat_length], (self.image_height, self.image_width)),\
            np.resize(image[flat_length:2*flat_length], (self.image_height, self.image_width)),\
            np.resize(image[2*flat_length:3*flat_length], (self.image_height, self.image_width))))
        else:
            out = np.resize(image, (self.image_height, self.image_width))
        return out

In [ ]:
import pickle
def loadPickle(path):
    
    with open(path, 'rb') as handle:
        return pickle.load(handle, encoding='latin1')

def normal_round(n):
    if n - np.floor(np.abs(n)) < 0.5:
        return np.floor(n)
    return np.ceil(n)

#i = int, r = round.
def ir(val):
    return int(normal_round(val))

def project(source, target, location, v=False):
    sh, sw = source.shape[:2]
    th, tw = target.shape[:2]
    
    #target frame
    y1 = max(0, ir(location[0] - sh/2.0))
    y2 = min(th, ir(location[0] + sh/2.0))
    x1 = max(0, ir(location[1] - sw/2.0))
    x2 = min(tw, ir(location[1] + sw/2.0))
    
    #source frame
    s_y1 = - ir(min(0, location[0] - sh/2.0 + 0.5))
    s_y2 = s_y1 + (y2 - y1)
    s_x1 = - ir(min(0, location[1] - sw/2.0 + 0.5))
    s_x2 = s_x1 + (x2 - x1)
    
    try: target[y1:y2, x1:x2] += source[s_y1:s_y2, s_x1:s_x2]
    except Exception as E:
        print(y1, y2, x1, x2)
        print(s_y1, s_y2, s_x1, s_x2)
        print(source.shape)
        print(target.shape)
        print(location)
        raise E
    
    if v:
        print(y1, y2, x1, x2)
        print(s_y1, s_y2, s_x1, s_x2)
        print(source.shape)
        print(target.shape)
        print(location)
    
    return target

def pad(img, padding, nans=False):
    size = ()
    for i in range(len(img.shape)):
        if i != 2: size += (img.shape[i] + 2*padding,)
        else: size += (img.shape[i],)

    out = np.zeros(size, dtype = img.dtype)
    
    if nans: out = np.full(size, np.nan)
    out[padding:-padding, padding:-padding] = img
    
    return out

class Retina:
    def __init__(self, gpu=True):
        self.loc = 0
        self.N = 0
        self.coeff = 0
        self.width = 0
        
        self._cudaRetina = CudaRetina() if gpu else None
        self._fixation = 0 #YX tuple
        self._imsize = 0
        self._gaussNorm = 0 #image
        self._gaussNormTight = 0 #image
        self._normFixation = 0 #YX tuple
        self._V = 0
        self._backproj = 0 #image
        self._backprojTight = 0 #image
        
    def info(self):
        print("loc - an Nx7 array containing retinal nodes defined as follows:\n\
    [x, y, d, angle (radians), dist_5, rf_sigma, rf_width]\n\
coeff - an array of variable size gaussian receptive field kernels\n\
V - the imagevector, output of retinal sampling\n\
gaussNorm - Gaussian normalization image for producing backprojections\n\
\n\
REMEMBER: all coordinates are tuples in the Y,X order, not X,Y.\n\
The only exception is the loc array\n\
REMEMBER2: coeff is redundantly wrapped in another matrix for backwards compatibility")
    
    def validate(self):
        assert(len(self.loc) == len(self.coeff[0]))
        if self._gaussNormTight is 0: self._normTight()
        
    def _normTight(self): 
        """Produce a tight-fitted Gaussian normalization image (width x width)"""
        GI = np.zeros((self.width, self.width))         
        r = self.width/2.0
        for i in range(self.N - 1, -1, -1): 
            GI = project(self.coeff[0,i], GI, self.loc[i,:2][::-1] + r)
        
        self._gaussNormTight = GI

    def loadLoc(self, path):
        self.loc = loadPickle(path)
        self.N = len(self.loc)
        self.width = 2*int(np.abs(self.loc[:,:2]).max() + self.loc[:,6].max()/2.0)
        
    def loadCoeff(self, path):
        self.coeff = loadPickle(path)
        
    def prepare(self, shape, fix):
        """Pre-compute fixation specific Gaussian normalization image """
        fix = (int(fix[0]), int(fix[1]))
        self.validate()
        self._normFixation = fix
        
        GI = np.zeros(shape[:2])
        GI = project(self._gaussNormTight, GI, fix)
        self._gaussNorm = GI
        if self._cudaRetina:
            self._cudaRetina.set_samplingfields(self.loc, self.coeff)
    
    def sample(self, image, fix):
        """Sample an image"""
        fix = (int(fix[0]), int(fix[1]))
        self.validate()
        self._fixation = fix
        # This will reset the image size only when it was changed.
        if self._imsize != image.shape[:2]:
            self._imsize = image.shape
        if self._cudaRetina:
            # TODO: helper function
            self._cudaRetina.image_width = image.shape[1]
            self._cudaRetina.image_height = image.shape[0]
            self._cudaRetina.rgb = len(image.shape) == 3 and image.shape[-1] == 3
            self._cudaRetina.center_x = int(fix[1])
            self._cudaRetina.center_y = int(fix[0])
            self._cudaRetina.set_gauss_norm(self._gaussNorm)
            V = self._cudaRetina.sample(np.uint8(image)) #XXX uint8 added
            self._V = V
            return V

        rgb = len(image.shape) == 3 and image.shape[-1] == 3
        p = self.width
        pic = pad(image, p, True)
        
        X = self.loc[:,0] + fix[1] + p
        Y = self.loc[:,1] + fix[0] + p
        
        if rgb: V = np.zeros((self.N,3))
        else: V = np.zeros((self.N))
        
        for i in range(0,self.N):
            w = self.loc[i,6]
            y1 = int(Y[i] - w/2+0.5)
            y2 = int(Y[i] + w/2+0.5)
            x1 = int(X[i] - w/2+0.5)
            x2 = int(X[i] + w/2+0.5)
            extract = pic[y1:y2,x1:x2]
            
            c = self.coeff[0, i]
            if rgb: kernel = np.dstack((c,c,c))
            else: kernel = c
            
            m = np.where(np.isnan(extract), 0, 1.0) #mask
            
            if rgb: f = 1.0/np.sum(m*kernel, axis = (0,1)) #TODO fix invalid value warnings
            else: f = 1.0/np.sum(m*kernel)
            
            extract = np.nan_to_num(extract)
            if rgb: V[i] = np.sum(extract*kernel, axis=(0,1)) * f
            else: V[i] = np.sum(extract*kernel) * f
       
        self._V = V
        return V